  

1.Data Loading and Preprocessing

   I had to read the dataset. 

   I needed to clean the data.

   I had to select features and target variables.

   I needed to convert data types.  

   I had to split the data into training, validation, and test sets.  

2.Parameter Initialization 

   I needed to initialize weight matrices and bias vectors. 

   I had to set appropriate dimensions for each layer.  

   I needed to use small random numbers for initialization.  

3.Forward Propagation 

   I had to compute the linear combination for each layer.

   I needed to apply activation functions.  

   I had to store intermediate values for backpropagation.  

4.Loss Function Calculation 

   I needed to compute the loss between predictions and true labels.  

   I had to choose an appropriate loss function.  

5.Backpropagation 

   I needed to calculate gradients using the chain rule.  

   I had to propagate errors backward through the network.  

6.Parameter Update  

   I needed to adjust weights and biases using gradients. 

   I had to apply the learning rate to updates.  

7.Epoch Loop  

   I needed to iterate over the entire dataset multiple times. 

   I had to shuffle data between epochs.  

8.Mini-batch Processing  

   I needed to split data into smaller batches.  

   I had to perform forward and backward passes per batch.  

9.Prediction 

   I had to make predictions on new data. 

   I needed to run forward propagation without training.  

10.Evaluation

   I needed to calculate performance metrics.  
   
   I had to compare results on validation and test sets.  



TensorFlow sample code implementations

Data Loading and Preprocessing:

 Done using pandas (pd.read_csv, filtering) and numpy (type conversion, reshaping) and sklearn (train_test_split) before the TensorFlow graph definition (Lines 14-32).

Parameter Initialization:

 Handled within the example_net function using tf.Variable(tf.random_normal(...)) for weights (weights) and biases (biases) (Lines 103-111). tf.Variable declares trainable parameters within the TensorFlow graph.

Forward Propagation:

 Defined symbolically within the example_net function using TensorFlow operations like tf.matmul (matrix multiplication), tf.add (or +), and tf.nn.relu (activation function) (Lines 113-117). The actual calculation happens later during sess.run(). The definition logits = example_net(X) (Line 120) connects the input placeholder X to the network structure.

Loss Function Calculation: 

Defined symbolically using tf.nn.sigmoid_cross_entropy_with_logits and tf.reduce_mean, assigned to the loss_op variable (Line 123). The calculation is performed during sess.run([loss_op, ...]).

Backpropagation: 

Implicitly handled by TensorFlow. When optimizer.minimize(loss_op) is defined (Line 126), TensorFlow automatically builds the necessary graph nodes to compute gradients.

Parameter Update :

Also implicitly handled by TensorFlow. Defining the optimizer (e.g., tf.train.AdamOptimizer, Line 125) and the train_op = optimizer.minimize(loss_op) (Line 126) sets up the mechanism. Running sess.run(train_op, ...) executes both backpropagation and the parameter updates according to the chosen optimizer's logic.

Epoch Loop: 

Implemented using a standard Python for loop: for epoch in range(num_epochs): (Line 136).

Mini-batch Processing: 

Achieved using the custom GetMiniBatch class (Lines 34-67) which acts as an iterator, and a nested Python for loop: for i, (mini_batch_x, mini_batch_y) in enumerate(get_mini_batch_train): (Line 141). Data is fed into the graph using the feed_dict argument in sess.run() (Lines 143, 144).

Prediction/Inference: 

The forward pass is run by calling sess.run(logits, feed_dict={X: ...}) or implicitly when calculating accuracy/loss on validation/test data (sess.run([loss_op, accuracy], feed_dict={X: X_val, Y: y_val}), Line 146). The actual predicted class would require applying tf.sigmoid and thresholding (or tf.argmax for multi-class).

Evaluation: 

Defined symbolically using tf.equal, tf.sign, tf.cast, tf.reduce_mean to create the accuracy operation (Lines 129-131). Calculated by running sess.run(accuracy, feed_dict={...}) on the appropriate data splits (Lines 144, 146, 148).

Placeholders: 

tf.placeholder (Lines 91, 92) defines inputs (X, Y) where actual data (mini-batches) will be fed during execution.

Initialization: 

tf.global_variables_initializer() (Line 134) creates an operation to initialize all tf.Variables. This must be run once at the beginning of the session using sess.run(init) (Line 137).

Session: 

tf.Session() (Line 136) creates the runtime environment to execute the graph operations.

In [ ]:
# -*- coding: utf-8 -*-
"""
Multi-class classification of Iris dataset (3 species) using neural network implementation in TensorFlow 
"""
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import MinMaxScaler
import tensorflow.compat.v1 as tf 
tf.disable_v2_behavior() 

import warnings
warnings.filterwarnings('ignore', category=FutureWarning)
tf.logging.set_verbosity(tf.logging.ERROR) 


try:
    df = pd.read_csv(r"C:\Users\Admin\Downloads\Iris.csv")
except FileNotFoundError:
    print("Error: Iris.csv not found.")
    exit()

y = df["Species"]
X = df.loc[:, ["SepalLengthCm", "SepalWidthCm", "PetalLengthCm", "PetalWidthCm"]]
X = np.array(X)
y_labels = np.array(y)
label_map = {"Iris-setosa": 0, "Iris-versicolor": 1, "Iris-virginica": 2}
y_integers = np.array([label_map[label] for label in y_labels]).reshape(-1, 1)

# 
X_train, X_temp, y_train_int, y_temp_int = train_test_split(
    X, y_integers, test_size=0.3, random_state=0, stratify=y_integers)
X_val, X_test, y_val_int, y_test_int = train_test_split(
    X_temp, y_temp_int, test_size=0.5, random_state=0, stratify=y_temp_int)

# 
enc = OneHotEncoder(handle_unknown='ignore', sparse=False)
y_train_one_hot = enc.fit_transform(y_train_int)
y_val_one_hot = enc.transform(y_val_int)
y_test_one_hot = enc.transform(y_test_int)

#
scaler = MinMaxScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_val_scaled = scaler.transform(X_val)
X_test_scaled = scaler.transform(X_test)

print("Data Shapes:")
print("X_train_scaled:", X_train_scaled.shape)
print("y_train_one_hot:", y_train_one_hot.shape)

class GetMiniBatch:
    
    def __init__(self, X, y, batch_size = 10, seed=0):
        self.batch_size = batch_size
        np.random.seed(seed)
        shuffle_index = np.random.permutation(np.arange(X.shape[0]))
        self.X = X[shuffle_index]
        self.y = y[shuffle_index]
        self._stop = np.ceil(X.shape[0]/self.batch_size).astype(np.int64)
    def __len__(self):
        return self._stop
    def __getitem__(self,item):
        p0 = item*self.batch_size
        p1 = item*self.batch_size + self.batch_size
        return self.X[p0:p1], self.y[p0:p1]
    def __iter__(self):
        self._counter = 0
        return self
    def __next__(self):
        if self._counter >= self._stop:
            raise StopIteration()
        p0 = self._counter*self.batch_size
        p1 = self._counter*self.batch_size + self.batch_size
        self._counter += 1
        return self.X[p0:p1], self.y[p0:p1]


learning_rate = 0.001
batch_size = 10
num_epochs = 100
n_hidden1 = 50
n_hidden2 = 100
n_input = X_train_scaled.shape[1]
n_samples = X_train_scaled.shape[0]
n_classes = y_train_one_hot.shape[1]

print("\nNetwork Parameters:")

tf.reset_default_graph()


X = tf.placeholder("float", [None, n_input])
Y = tf.placeholder("float", [None, n_classes])

get_mini_batch_train = GetMiniBatch(X_train_scaled, y_train_one_hot, batch_size=batch_size)

def example_net(x):
    
    initializer = tf.glorot_uniform_initializer() 

    weights = {
        'w1': tf.Variable(initializer([n_input, n_hidden1])),
        'w2': tf.Variable(initializer([n_hidden1, n_hidden2])),
        'w3': tf.Variable(initializer([n_hidden2, n_classes]))
    }
    biases = {
        'b1': tf.Variable(tf.zeros([n_hidden1])),
        'b2': tf.Variable(tf.zeros([n_hidden2])),
        'b3': tf.Variable(tf.zeros([n_classes]))
    }

    layer_1 = tf.add(tf.matmul(x, weights['w1']), biases['b1'])
    layer_1 = tf.nn.relu(layer_1)
    layer_2 = tf.add(tf.matmul(layer_1, weights['w2']), biases['b2'])
    layer_2 = tf.nn.relu(layer_2)
    layer_output = tf.matmul(layer_2, weights['w3']) + biases['b3']
    return layer_output


logits = example_net(X)


loss_op = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(labels=Y, logits=logits))


optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
train_op = optimizer.minimize(loss_op)


correct_pred = tf.equal(tf.argmax(logits, axis=1), tf.argmax(Y, axis=1))


accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))


init = tf.global_variables_initializer()

#

with tf.Session() as sess:
    sess.run(init) 

    for epoch in range(num_epochs):
        total_loss = 0
        
        for i, (mini_batch_x, mini_batch_y) in enumerate(get_mini_batch_train):
            sess.run(train_op, feed_dict={X: mini_batch_x, Y: mini_batch_y})
            loss = sess.run(loss_op, feed_dict={X: mini_batch_x, Y: mini_batch_y})
            total_loss += loss * mini_batch_x.shape[0]

        avg_train_loss = total_loss / n_samples
        val_loss, val_acc = sess.run([loss_op, accuracy], feed_dict={X: X_val_scaled, Y: y_val_one_hot})
        print("Epoch {:>3}, Train Loss: {:.4f}, Val Loss: {:.4f}, Val Acc: {:.3f}".format(
            epoch, avg_train_loss, val_loss, val_acc))

    # 
    
    test_acc = sess.run(accuracy, feed_dict={X: X_test_scaled, Y: y_test_one_hot})
    print("-----")
    print("Final Test Accuracy: {:.3f}".format(test_acc))
    

Data Shapes:
X_train_scaled: (105, 4)
y_train_one_hot: (105, 3)

Network Parameters:
Epoch   0, Train Loss: 1.0326, Val Loss: 0.9705, Val Acc: 0.591
Epoch   1, Train Loss: 0.9255, Val Loss: 0.8769, Val Acc: 0.682
Epoch   2, Train Loss: 0.8352, Val Loss: 0.7885, Val Acc: 0.682
Epoch   3, Train Loss: 0.7465, Val Loss: 0.7010, Val Acc: 0.682
Epoch   4, Train Loss: 0.6619, Val Loss: 0.6216, Val Acc: 0.682
Epoch   5, Train Loss: 0.5863, Val Loss: 0.5536, Val Acc: 0.773
Epoch   6, Train Loss: 0.5241, Val Loss: 0.4993, Val Acc: 0.864
Epoch   7, Train Loss: 0.4760, Val Loss: 0.4582, Val Acc: 0.909
Epoch   8, Train Loss: 0.4391, Val Loss: 0.4253, Val Acc: 0.955
Epoch   9, Train Loss: 0.4086, Val Loss: 0.3953, Val Acc: 0.955
Epoch  10, Train Loss: 0.3815, Val Loss: 0.3683, Val Acc: 0.955
Epoch  11, Train Loss: 0.3558, Val Loss: 0.3413, Val Acc: 0.955
Epoch  12, Train Loss: 0.3307, Val Loss: 0.3174, Val Acc: 0.955
Epoch  13, Train Loss: 0.3065, Val Loss: 0.2936, Val Acc: 0.955
Epoch  14, Train Lo

In [6]:

"""
Regression of House Prices dataset using neural network implementation in TensorFlow 1.x

"""
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
import tensorflow.compat.v1 as tf 
tf.disable_v2_behavior() 

import warnings


warnings.filterwarnings('ignore', category=FutureWarning)
tf.logging.set_verbosity(tf.logging.ERROR) 


try:
    df = pd.read_csv(r"C:\Users\Admin\Downloads\train.csv")
except FileNotFoundError:
    print("Error: train.csv House Prices dataset not found.")
    exit()

# 
features = ["GrLivArea", "YearBuilt"]
target = "SalePrice"

df_clean = df[features + [target]].dropna()

X = df_clean[features].values
y = df_clean[target].values


y_log = np.log(y).astype(np.float32) 


y_log = y_log.reshape(-1, 1)


X = X.astype(np.float32)


X_train, X_temp, y_train_log, y_temp_log = train_test_split(
    X, y_log, test_size=0.3, random_state=0) 
X_val, X_test, y_val_log, y_test_log = train_test_split(
    X_temp, y_temp_log, test_size=0.5, random_state=0) 


scaler = MinMaxScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_val_scaled = scaler.transform(X_val)
X_test_scaled = scaler.transform(X_test)

print("Data Shapes:")
print("X_train_scaled:", X_train_scaled.shape)
print("y_train_log:", y_train_log.shape)
print("X_val_scaled:", X_val_scaled.shape)
print("y_val_log:", y_val_log.shape)
print("X_test_scaled:", X_test_scaled.shape)
print("y_test_log:", y_test_log.shape)


class GetMiniBatch:
    def __init__(self, X, y, batch_size = 10, seed=0):
        self.batch_size = batch_size
        np.random.seed(seed)
        shuffle_index = np.random.permutation(np.arange(X.shape[0]))
        self.X = X[shuffle_index]
        self.y = y[shuffle_index]
        self._stop = np.ceil(X.shape[0]/self.batch_size).astype(np.int64)
    def __len__(self):
        return self._stop
    def __getitem__(self,item):
        p0 = item*self.batch_size
        p1 = item*self.batch_size + self.batch_size
        return self.X[p0:p1], self.y[p0:p1]
    def __iter__(self):
        self._counter = 0
        return self
    def __next__(self):
        if self._counter >= self._stop:
            raise StopIteration()
        p0 = self._counter*self.batch_size
        p1 = self._counter*self.batch_size + self.batch_size
        self._counter += 1
        return self.X[p0:p1], self.y[p0:p1]


learning_rate = 0.001
batch_size = 16     
num_epochs = 100  

n_hidden1 = 50
n_hidden2 = 100
n_input = X_train_scaled.shape[1] 
n_samples = X_train_scaled.shape[0]
n_classes = 1 


print("Input features:", n_input)
print("Hidden Layer 1 Nodes:", n_hidden1)
print("Hidden Layer 2 Nodes:", n_hidden2)
print("Output Nodes:", n_classes) 
print("Learning Rate:", learning_rate)
print("Batch Size:", batch_size)
print("Epochs:", num_epochs)


tf.reset_default_graph()

#
X = tf.placeholder("float", [None, n_input])
Y = tf.placeholder("float", [None, n_classes]) 


get_mini_batch_train = GetMiniBatch(X_train_scaled, y_train_log, batch_size=batch_size)

# 
def example_net(x):
    initializer = tf.glorot_uniform_initializer()
    weights = {
        'w1': tf.Variable(initializer([n_input, n_hidden1])),
        'w2': tf.Variable(initializer([n_hidden1, n_hidden2])),
        'w3': tf.Variable(initializer([n_hidden2, n_classes])) 
    }
    biases = {
        'b1': tf.Variable(tf.zeros([n_hidden1])),
        'b2': tf.Variable(tf.zeros([n_hidden2])),
        'b3': tf.Variable(tf.zeros([n_classes])) 
    }
    layer_1 = tf.add(tf.matmul(x, weights['w1']), biases['b1'])
    layer_1 = tf.nn.relu(layer_1)
    layer_2 = tf.add(tf.matmul(layer_1, weights['w2']), biases['b2'])
    layer_2 = tf.nn.relu(layer_2)
    # 
    layer_output = tf.matmul(layer_2, weights['w3']) + biases['b3']
    return layer_output

# 
logits = example_net(X)

# 
loss_op = tf.reduce_mean(tf.square(Y - logits))

rmse_op = tf.sqrt(loss_op)


optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
train_op = optimizer.minimize(loss_op)

init = tf.global_variables_initializer()

with tf.Session() as sess:
    sess.run(init) 

    for epoch in range(num_epochs):
        total_loss = 0
        for i, (mini_batch_x, mini_batch_y) in enumerate(get_mini_batch_train):
            
            sess.run(train_op, feed_dict={X: mini_batch_x, Y: mini_batch_y})
            
            loss = sess.run(loss_op, feed_dict={X: mini_batch_x, Y: mini_batch_y})
            total_loss += loss * mini_batch_x.shape[0] 

        avg_train_loss = total_loss / n_samples

        #
        val_loss, val_rmse = sess.run([loss_op, rmse_op], feed_dict={X: X_val_scaled, Y: y_val_log})

        
        print("Epoch {:>3}, Train Loss (MSE): {:.4f}, Val Loss (MSE): {:.4f}, Val RMSE: {:.4f}".format(
            epoch, avg_train_loss, val_loss, val_rmse))

    
    test_loss, test_rmse = sess.run([loss_op, rmse_op], feed_dict={X: X_test_scaled, Y: y_test_log})
    
    # Report final test metrics
    print("Final Test Loss (MSE): {:.4f}".format(test_loss))
    print("Final Test RMSE:       {:.4f}".format(test_rmse))
    

Data Shapes:
X_train_scaled: (1022, 2)
y_train_log: (1022, 1)
X_val_scaled: (219, 2)
y_val_log: (219, 1)
X_test_scaled: (219, 2)
y_test_log: (219, 1)
Input features: 2
Hidden Layer 1 Nodes: 50
Hidden Layer 2 Nodes: 100
Output Nodes: 1
Learning Rate: 0.001
Batch Size: 16
Epochs: 100
Epoch   0, Train Loss (MSE): 114.7908, Val Loss (MSE): 63.8812, Val RMSE: 7.9926
Epoch   1, Train Loss (MSE): 16.6778, Val Loss (MSE): 2.8445, Val RMSE: 1.6866
Epoch   2, Train Loss (MSE): 2.3029, Val Loss (MSE): 1.8991, Val RMSE: 1.3781
Epoch   3, Train Loss (MSE): 1.4513, Val Loss (MSE): 1.0761, Val RMSE: 1.0374
Epoch   4, Train Loss (MSE): 0.7478, Val Loss (MSE): 0.5011, Val RMSE: 0.7079
Epoch   5, Train Loss (MSE): 0.3439, Val Loss (MSE): 0.2216, Val RMSE: 0.4707
Epoch   6, Train Loss (MSE): 0.1656, Val Loss (MSE): 0.1081, Val RMSE: 0.3289
Epoch   7, Train Loss (MSE): 0.0935, Val Loss (MSE): 0.0634, Val RMSE: 0.2519
Epoch   8, Train Loss (MSE): 0.0654, Val Loss (MSE): 0.0461, Val RMSE: 0.2146
Epoch   9, 

In [7]:

"""
MNIST Digit Classification using neural network implementation in TensorFlow 
"""
import numpy as np
import pandas as pd 
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
import tensorflow.compat.v1 as tf 
tf.disable_v2_behavior() 
from tensorflow.keras.datasets import mnist
import warnings

warnings.filterwarnings('ignore', category=FutureWarning)
tf.logging.set_verbosity(tf.logging.ERROR) 

(X_train_orig, y_train_orig), (X_test, y_test) = mnist.load_data()

print("Original MNIST shapes:")
print("X_train_orig:", X_train_orig.shape) 
print("y_train_orig:", y_train_orig.shape) 
print("X_test:", X_test.shape)       
print("y_test:", y_test.shape)      

X_train_flat = X_train_orig.reshape(-1, 784)
X_test_flat = X_test.reshape(-1, 784)

X_train_norm = (X_train_flat / 255.0).astype(np.float32)
X_test_norm = (X_test_flat / 255.0).astype(np.float32)

y_train_col = y_train_orig.reshape(-1, 1).astype(np.int64) 
y_test_col = y_test.reshape(-1, 1).astype(np.int64)   


X_train, X_val, y_train_int, y_val_int = train_test_split(
    X_train_norm, y_train_col, test_size=0.2, random_state=0, stratify=y_train_col) 

enc = OneHotEncoder(handle_unknown='ignore', sparse=False)
y_train_one_hot = enc.fit_transform(y_train_int)
y_val_one_hot = enc.transform(y_val_int)
y_test_one_hot = enc.transform(y_test_col)



print("X_train:", X_train.shape) 
print("y_train_one_hot:", y_train_one_hot.shape) 
print("X_val:", X_val.shape)  
print("y_val_one_hot:", y_val_one_hot.shape) 
print("X_test_norm:", X_test_norm.shape)   
print("y_test_one_hot:", y_test_one_hot.shape) 


# 
class GetMiniBatch:
    
    def __init__(self, X, y, batch_size = 10, seed=0):
        self.batch_size = batch_size
        np.random.seed(seed)
        shuffle_index = np.random.permutation(np.arange(X.shape[0]))
        self.X = X[shuffle_index]
        self.y = y[shuffle_index]
        self._stop = np.ceil(X.shape[0]/self.batch_size).astype(np.int64)
    def __len__(self):
        return self._stop
    def __getitem__(self,item):
        p0 = item*self.batch_size
        p1 = item*self.batch_size + self.batch_size
        return self.X[p0:p1], self.y[p0:p1]
    def __iter__(self):
        self._counter = 0
        return self
    def __next__(self):
        if self._counter >= self._stop:
            raise StopIteration()
        p0 = self._counter*self.batch_size
        p1 = self._counter*self.batch_size + self.batch_size
        self._counter += 1
        return self.X[p0:p1], self.y[p0:p1]

# 
learning_rate = 0.001
batch_size = 128     
num_epochs = 20      

# 
n_hidden1 = 256      
n_hidden2 = 128      
n_input = X_train.shape[1] 
n_samples = X_train.shape[0] 
n_classes = y_train_one_hot.shape[1] 

print("Input features:", n_input)
print("Hidden Layer 1 Nodes:", n_hidden1)
print("Hidden Layer 2 Nodes:", n_hidden2)
print("Output Classes:", n_classes)
print("Learning Rate:", learning_rate)
print("Batch Size:", batch_size)
print("Epochs:", num_epochs)

# 
tf.reset_default_graph()

# 
X = tf.placeholder("float", [None, n_input]) 
Y = tf.placeholder("float", [None, n_classes])

# 
get_mini_batch_train = GetMiniBatch(X_train, y_train_one_hot, batch_size=batch_size)

# 
def example_net(x):
    initializer = tf.glorot_uniform_initializer()
    weights = {
        'w1': tf.Variable(initializer([n_input, n_hidden1])),
        'w2': tf.Variable(initializer([n_hidden1, n_hidden2])),
        'w3': tf.Variable(initializer([n_hidden2, n_classes]))
    }
    biases = {
        'b1': tf.Variable(tf.zeros([n_hidden1])),
        'b2': tf.Variable(tf.zeros([n_hidden2])),
        'b3': tf.Variable(tf.zeros([n_classes]))
    }

    layer_1 = tf.add(tf.matmul(x, weights['w1']), biases['b1'])
    layer_1 = tf.nn.relu(layer_1)
    layer_2 = tf.add(tf.matmul(layer_1, weights['w2']), biases['b2'])
    layer_2 = tf.nn.relu(layer_2)
    #
    layer_output = tf.matmul(layer_2, weights['w3']) + biases['b3']
    return layer_output

# 
logits = example_net(X)

#
loss_op = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(labels=Y, logits=logits))

# 
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
train_op = optimizer.minimize(loss_op)

#
correct_pred = tf.equal(tf.argmax(logits, axis=1), tf.argmax(Y, axis=1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

# 
init = tf.global_variables_initializer()

with tf.Session() as sess:
    sess.run(init) 

    for epoch in range(num_epochs):
        total_loss = 0

        for i, (mini_batch_x, mini_batch_y) in enumerate(get_mini_batch_train):
            sess.run(train_op, feed_dict={X: mini_batch_x, Y: mini_batch_y})
            loss = sess.run(loss_op, feed_dict={X: mini_batch_x, Y: mini_batch_y})
            total_loss += loss * mini_batch_x.shape[0] 

        avg_train_loss = total_loss / n_samples

        # 
        val_loss, val_acc = sess.run([loss_op, accuracy], feed_dict={X: X_val, Y: y_val_one_hot})

        print("Epoch {:>3}, Train Loss: {:.4f}, Val Loss: {:.4f}, Val Acc: {:.4f}".format( 
            epoch, avg_train_loss, val_loss, val_acc))

    #
    #
    test_acc = sess.run(accuracy, feed_dict={X: X_test_norm, Y: y_test_one_hot})
    print("Final Test Accuracy: {:.4f}".format(test_acc)) 

Original MNIST shapes:
X_train_orig: (60000, 28, 28)
y_train_orig: (60000,)
X_test: (10000, 28, 28)
y_test: (10000,)
X_train: (48000, 784)
y_train_one_hot: (48000, 10)
X_val: (12000, 784)
y_val_one_hot: (12000, 10)
X_test_norm: (10000, 784)
y_test_one_hot: (10000, 10)
Input features: 784
Hidden Layer 1 Nodes: 256
Hidden Layer 2 Nodes: 128
Output Classes: 10
Learning Rate: 0.001
Batch Size: 128
Epochs: 20
Epoch   0, Train Loss: 0.2793, Val Loss: 0.1464, Val Acc: 0.9572
Epoch   1, Train Loss: 0.1036, Val Loss: 0.1089, Val Acc: 0.9665
Epoch   2, Train Loss: 0.0628, Val Loss: 0.0939, Val Acc: 0.9718
Epoch   3, Train Loss: 0.0411, Val Loss: 0.0913, Val Acc: 0.9721
Epoch   4, Train Loss: 0.0272, Val Loss: 0.0914, Val Acc: 0.9718
Epoch   5, Train Loss: 0.0189, Val Loss: 0.0899, Val Acc: 0.9737
Epoch   6, Train Loss: 0.0130, Val Loss: 0.0922, Val Acc: 0.9757
Epoch   7, Train Loss: 0.0104, Val Loss: 0.0892, Val Acc: 0.9777
Epoch   8, Train Loss: 0.0088, Val Loss: 0.0989, Val Acc: 0.9764
Epoch  